# Project 2: The three-body problem<br/>*solving a chaotic motion problem using deep neural networks*
## Learning from data [TIF285], Chalmers, Fall 2024

Last revised: 9-Oct-2024 by Christian Forssén [christian.forssen@chalmers.se]

## Instructions

- See deadline on the course web page
- This project is performed in groups of two students. 
- The second part of the project is optional. See examination rules on the course web page.
- Hand-in your written report and your solution source code via Canvas. 
- Please also upload your ANN model with the trained parameters as `myNN.keras` using the keras `save_model` method: `tf.keras.models.save_model(model, 'myNN.keras')`, which employs the native Keras file format.

### Written report
- Page limit: 6 pages (excluding title page and list of references). 3 extra pages are allowed when doing also the optional extra task.
- Give a short description of the nature of the problem and the methods you have used.
- Include your results either in figure form or in a table. All tables and figures should have relevant captions and labels on the axes.
- Try to give an interpretation of you results.
- Upload the source code of your program as a separate file (.ipynb or .py). Comment your program properly.

### Learning goals:
* Apply deep neural networks to build an emulator for a physics problem that is difficult and costly to solve numerically.
* Get familiar with [tensorflow](https://www.tensorflow.org/), the popular open-source library to train ML models created by the Google Brain team.
* Get experience with some diagnostics used in the training of neural networks. 
* Understand the difficulties and choices involved when training a deep neural network.
* This is a less-guided set of tasks and you will have to put together ideas and tools we've discussed.
* Reproduce scientific results from a published paper.

## Main task
The overall project goal is to reproduce various results in a paper: [*Newton vs the machine: solving the chaotic three-body problem using deep neural networks*](https://arxiv.org/abs/1910.07291) by Philip G. Breen, Christopher N. Foley, Tjarda Boekholt, Simon Portegies Zwart.

The authors have graciously agreed that we use the raw data from their study for this project (but please don't distribute any further).

### Physical problem

In this project we are studying the motion of three bodies of equal mass that move under the mutual influence of classical gravitational forces. Due to the system's chaotic nature, the solution to this problem for a given initialization can only be found by performing iterative numerical calculations that have potentially very large computational cost. 

Here you will demonstrate that a deep artificial neural network (ANN) is able to learn the solutions to the equations of motion from a set of numerical training data. This ANN can then provide good approximations for new initial conditions at a predictable (and small) computational cost.

The training data that we will be using was generated with a state-of-the-art numerical code `Brutus` [[Boekholt, T., Portegies Zwart, S. On the reliability of N-body simulations. Comput. Astrophys. 2, 2 (2015).](https://doi.org/10.1186/s40668-014-0005-3)]. The run time of that code for the trajectories studied here ranges from minutes up to hours. In the end, it will therefore be relevant to quantify what is the computational cost of making a prediction with our constructed ANN.

### Initial condition and choice of coordinates
- We work in the the center-of-mass (CM) frame and only consider the scenario in which the three masses start from rest.
- It can be shown that, without loss of generality, it is always possible to choose a coordinate system such that the three particles $p_1, p_2, p_3$ move in a plane so that just $(x,y)$ coordinates are needed.
- Furthermore, the $(x,y)$ coordinates can be chosen and scaled such that particle 1 starts in position $(1,0)$ and the initial position of particle 2 is located in the second quantile ($x<0$, $y>0$).
- Since we are in the CM frame and our system consists of three equal masses we will always have $x_1+x_2+x_3=0$ and $y_1+y_2+y_3=0$. Consequently, we can determine the position of particle 3 from knowledge of the positions of particles 1 and 2.

See also Fig. 1 in the paper.

### Data
The data that you should use is made available as a compressed numpy file on the Canvas course page. Note that, due to its size, it is not included in the `tif285` git repo.

Navigate to Files > Project2  and download `data_project2.npz` into your work directory.

In [1]:
# Load modules
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# Add your own import statements if needed
import time

2024-10-25 21:26:51.006227: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-25 21:26:51.033034: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load and unpack a compressed npy array
load_data = np.load('data_project2.npz')
data = load_data['arr_0']
print(data.shape)

(9000, 1000, 9)


- The first axis labels the 9000 data samples.
- The second axis is the 1000 time steps, which corresponds to 3.9 time units. The original data is composed of 2561 time steps, reaching 10 time units, but as discussed in the paper the training of the ANN works better for the somewhat shorter time interval.
- The nine columns of the last axis correspond to: $[t, x_1, y_1, x_2, y_2, v_{x,1}, v_{y,1}, v_{x,2}, v_{y,2}]$ (note again that the position of particle 3 can be obtained from the positions of particles 1 and 2). The velocity data is only used in the extra task.
- Some of the trajectories contain collisions such that the positions of the particles get stuck at (0, 0). Check the data to find those trajectories and remove them before training your model.
- Note that the trajectories are represented by data points in sets of 1000 (time steps). While creating batches of data, make sure you are not splitting data from one trajectory into different batches.

### List of tasks
1. Download and load the pre-trained ANN from the paper (see below) and construct/train your own ANN using the provided data. Compare the complexity (number of trainable parameters) in the pre-trained versus your own ANN.
1. Show the Mean Absolute Error (MAE) vs epoch from the training of your own ANN (as in Fig. 3 of the paper). I.e. you should show both the MAE metric on the training and the validation data.
1. Check the accuracy of the pre-trained ANN by visually comparing some trajectories with the ANN predictions as in Fig. 4 of the paper. Add also the trajectory of your ANN and comment on its performance. (Note that you should not expect your own ANN to perform as well as the pre-trained one. In addition, the trajectories that are shown in the paper seem to be ones for which the model works very well).
1. Check the computational cost of making a prediction with the ANN. You will find that it is much shorter than the numerical integration performed by `Brutus` (which according to the reference takes minutes to hours for finding a trajectory).
1. Having access to these ANN emulators you can explore the chaotic aspect of this motion by creating a hundred different trajectories from a slightly disturbed initial condition. Try to reproduce Fig. 5 in the paper (both with the pre-trained and your own ANN).

### Suggestions for how to proceed:
* Follow the lead of the demo notebook for using tensorflow / keras (plus see the hints below).
* First download and load the data set and the ANN from the paper (both available on the Canvas course page. Navigate to Files > Project2; see hint below)
* It is suggested (but not required) to create helper functions (see hints below): 
  * `get_data` for extracting a trajectory (positions of particle 1 as a function of the discrete time) from the data; 
  * `get_trajectories` for generating the positions of all particles from a prediction output that contains the arrays of the first two particles' positions; 
  * `plot_trajectories` for plotting a trajectory.
* When creating your own ANN it is suggested that you first design a smaller one than in the paper (e.g., using five dense layers with 64 neurons in each) and analyse how it performs by looking at the training vs validation error. The creation of a large network with ten layers and 128 nodes per layer is optional (and might require more cpu-hours for training).
* Try first with the `relu` activation function and the `Adam` optimizer with a learning rate of 0.001 and moments of 0.5 (these hyper-parameters can easily be changed if you want; see hints below). 
    - The choice of optimizer  and hyper-parameters such as learning rate can certainly influence your results. For the interested, an extensive guide with open-source experimental results is published here: https://arxiv.org/abs/2007.01547.
* Use 90% of the data for training and 10% for validation. You probably need at least hundred epochs for training and it is suggested to split the data into 10-100 batches (see hints below).

### Comments and hints
* Download the pre-trained ANN model `Breen_NN_project2.keras` and load it with: `pre_trained_model = keras.models.load_model("Breen_NN_project2.keras")`. This file should also be included in the gzipped tarball that contains the project 2 files.
  *Please let us know if you have difficulties loading this model as there might be compatibility issues between different `tensorflow` versions. As a backup, the weights of Breen's model are stored in `Breen_NN_project2.weights.h5` and can be loaded using the `load_weights()` method into a network of the correct topology (10 hidden layers of width 128; relu activation).*
* The ANN input should be an array with three elements: $[t,x_2,y_2]$, where the last two will be the initial position of particle 2 and the first one is the time step for which the particle positions should be predicted. Remember that particle 1 always starts in $(1,0)$. 
* Input data corresponding to a single trajectory is then of shape $1000 \times 3$ where the first column contains the time steps while the $x_2,y_2$ columns are repetitions of the initial position.
* I.e., the inputs to the `tensorflow` network is a tensor with three columns where the second and third columns have values that are repeated (with values of the initial position at [t=0]) for all the rows corresponding to a single trajectory.
* The output should be an array with four elements $[x_1,y_1,x_2,y_2]$ that corresponds to the positions of particles 1 and 2 at the time step $t$ that was given in the first position of the input. Again, we can consider a tensor (batch) of outputs corresponding, e.g., to a full trajectory.
* As mentioned, some of the trajectories contain collisions such that the positions of the particles get stuck at (0, 0). Check the data to find those trajectories and remove them before training your model.
* The data can be turned into a `tensorflow` dataset using: `dataset = tf.data.Dataset.from_tensor_slices((x, y))`, where `x` and `y` are input and output data, respectively (you might possibly need to reshape using `x.reshape(-1, 3)` and `y.reshape(-1, 4)`)
* Having created a dataset, e.g. `train_dataset`, you can make batches (e.g. with batchsize 5000) with `train_dataset = train_dataset.batch(5000)`. The same should then be done for the validation data. As mentioned, the trajectories are represented by data points in sets of 1000 (time steps). While creating batches of data, make sure you are not mixing part of the data from one trajectory with a different trajectory. 
* When compiling a `tensorflow` model, with `model.compile()`, the Adam optimizer can be specified via the argument `optimizer=keras.optimizers.Adam(0.001, 0.5, 0.5)`
* Furthermore, the loss function can be chosen using the argument `loss=keras.losses.MeanAbsoluteError()`
* Fitting the model for 100 epochs with specified training and validation data sets can be performed with: `model.fit(train_dataset, epochs=100, validation_data = val_dataset)`
* Given the cost of training the network with all data, it is not required to make extensive tests of tuning the hyperparameters.
* More exact timing of a function can be measured using ["Jupyter notebook magic"](https://towardsdatascience.com/the-top-5-magic-commands-for-jupyter-notebooks-2bf0c5ae4bb8) by putting the command to be measured in a notebook cell that starts with `%timeit`. This will run the cell many times and report the mean and standard deviation of the evaluation time.

#### Helper function docstrings
The following are just suggestions. You are allowed to construct a solution using your own helper functions.

In [3]:
#importing existing model
pre_trained_model = keras.models.load_model("Breen_NN_project2.keras")
pre_trained_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_78 (Dense)                │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_80 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_84 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_86 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_87 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_88 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 149,636 (1.14 MB)

 Trainable params: 149,636 (1.14 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
def trajectory_checker(): #checks whether the particles get stuck at origin for all trajectories in data
    
    data_res = []
    
    valid_trajectory_flag = True #flag to designate whether a trajectory is valid or faulty (stuck)
    
    c1 = 0 #counters for identifying faulty trajectories
    c2 = 0
    c3 = 0
    
    for i in data: #looping through trajectories
        valid_trajectory_flag = True
        c2 = 0
        for j in i: #looping through points in trajectory i
            
            x1 = j[1] #parsing coordinates for a specific time data entry
            y1 = j[2]
            x2 = j[3]
            y2 = j[4]
            x3 = -x1-x2
            y3 = -y1-y2
            
            x_equality = (x1 == x2) and (x2 == x3) and (x3 == 0) #validity checking (all particles stuck at origin)
            y_equality = (y1 == y2) and (y2 == y3) and (y3 == 0)
            

            if(x_equality and y_equality): #if all particles are stuck
                #print(c1,c2)
                #print(x1,x2,x3)
                #print(y1,y2,y3)
                c3 += 1
                valid_trajectory_flag = False #invalidate trajectory
                break
            
            c2 += 1
        if(valid_trajectory_flag): #if i is a valid (unstuck) trajectory, add to resultant dataset
            data_res.append(i)
        c1 += 1
    #print(c3)
    return np.array(data_res)

In [5]:
data = trajectory_checker()

In [6]:
    def get_data(idx):
        """
        Get one training instance from the data set at the index idx. 
        
        The data is assumed to be in an array `data`.
        
        Args:
            idx (int): An integer specifying which of the training example to fetch
            
        Returns:
            x (array): An array of shape (time_steps, 3) which specifies the input to
                       the neural network. The first column is the time and the second
                       and third columns specify the (x, y) coordinates of the second
                       particle. Note that the first particle is always assumed to be
                       at (1, 0) and the third particle can be inferred from the first
                       and second particle's position.
                       
            y (array): An array of shape (time_steps, 4) which specifies the output that 
                       is expected from the neural network.
                       
                       The first two columns specify the (x, y) coordinates of the first
                       particles and the next two columns give the coordinates of the 
                       second particle for the specified time (length of the columns).
                       The third particles position can be inferred from the first
                       and second particle's position.
        """
        y = 0
        
        data_instance = data[idx] #trajectory at idx

        #format : [t, x_1, y_1, x_2, y_2, v_{x,1}, v_{y,1}, v_{x,2}, v_{y,2}]
        t = data_instance[:,0]
        x1 = data_instance[:,1]
        y1 = data_instance[:,2]
        x2 = data_instance[:,3]
        y2 = data_instance[:,4]

        x2_init = np.array([x2[0]] * len(x2))#initial 2nd particle position
        y2_init = np.array([y2[0]] * len(y2))
        
        x = np.column_stack((t,x2_init,y2_init)) #input params
        y = np.column_stack((x1,y1,x2,y2)) #output params
        
        return x,y
        #raise NotImplementedError

In [7]:
def get_trajectories(pred):
    """
    Gets the trajectories from a predicted output pred.
    
    Args:
        pred (array): An array of shape (N, 4) where N is the number of time
                      steps. The four columns give the positions of the particles
                      1 and 2 for all the time steps.
    Returns:
        p1, p2, p3 (tuple of arrays): Three arrays of dimensions (N, 2) where N is the number 
                             of time steps and the two columns for each array give 
                             the positions of the three particles (p1, p2, p3)
    """
    x1 = pred[:,0]
    y1 = pred[:,1]
    x2 = pred[:,2]
    y2 = pred[:,3]
    x3 = -x1-x2 #cm frame of reference
    y3 = -y1-y2
    
    p1 = np.column_stack((x1,y1))
    p2 = np.column_stack((x2,y2))
    p3 = np.column_stack((x3,y3))

    return (p1,p2,p3)
    #raise NotImplementedError

In [8]:
def plot_trajectories(p1, p2, p3, ax=None, **kwargs):
    """
    Plots trajectories for points p1, p2, p3
    
    Args:
        p1, p2, p3 (array): Three arrays each of shape (n, 2) where n is the number
                            of time steps. Each array is the (x, y) position for the
                            particles
        ax (axis object): Default None, in which case a new axis object is created.
        kwargs (dict): Optional keyword arguments for plotting
        
    Returns:
        ax: Axes object
    """
    if ax is None:
        fig, ax = plt.subplots()
        
    x1 = p1[:,0]
    y1 = p1[:,1]
    x2 = p2[:,0]
    y2 = p2[:,1]
    x3 = p3[:,0]
    y3 = p3[:,1]

    ax.plot(x1, y1, label="p1", **kwargs)
    ax.plot(x2, y2, label="p2", **kwargs)
    ax.plot(x3, y3, label="p3", **kwargs)

    ax.set_xlabel('X position')
    ax.set_ylabel('Y position')
    ax.legend()
    
    return ax
    #raise NotImplementedError

In [9]:
#data initialization
#splitting into input and output parameters

data_x = []
data_y = []

for i in range(data.shape[0]): #extracting x and y for each trajectory
    x,y = get_data(i)
    data_x.append(x) #appending x and y to a set
    data_y.append(y)

data_x = np.array(data_x)
data_y = np.array(data_y)

In [10]:
#splitting into training and validation datasets
split_index = int(0.9 * data.shape[0]) #9:1 ratio of train:val

data_x_train = data_x[:split_index]
data_y_train = data_y[:split_index]
data_x_val = data_x[split_index:]
data_y_val = data_y[split_index:]

#note: all training data should be in a single array and model fitting must be done in a single fit function
#iterating over all trajectories and performing individual fits will end up in an ann that is fitted only to the last trajectory
#learnt this the hard way with over 7 hours wasted on training absolute garbage
#remove this comment for submission

data_x_train = np.row_stack(data_x_train)
data_y_train = np.row_stack(data_y_train)
data_x_val = np.row_stack(data_x_val)
data_y_val = np.row_stack(data_y_val)

trajectory_length = 1000
batch_size = 2*trajectory_length #batch size must be an integer multiple of trajectory length to avoid mixing trajectory data points

#print(data_x_train.shape)
#print(data_y_train.shape)
#print(data_x_val.shape)
#print(data_y_val.shape)

In [11]:
#converting data into a tensorflow dataset
dataset_train = tf.data.Dataset.from_tensor_slices((data_x_train, data_y_train))
dataset_val = tf.data.Dataset.from_tensor_slices((data_x_val,data_y_val))

dataset_train = dataset_train.batch(batch_size)
dataset_val = dataset_val.batch(batch_size)

In [12]:
#model synthesis
three_body_model = keras.models.Sequential([
    keras.layers.InputLayer(shape=(3,)),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(4,activation='linear')]
)
#NOTE: the final layer must have a linear activation function because we are defining this output for real valued parameters
#and not for probabilistic outputs.

In [13]:
three_body_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 149,636 (584.52 KB)

 Trainable params: 149,636 (584.52 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
#model compilation
three_body_model.compile(optimizer=keras.optimizers.Adam(0.001, 0.5, 0.5),
              loss=keras.losses.MeanAbsoluteError(),
              metrics=['accuracy'])

In [15]:
#training and validation
three_body_model.fit(dataset_train, epochs=500, validation_data=dataset_val)

Epoch 1/500
3430/3430 ━━━━━━━━━━━━━━━━━━━━ 39s 11ms/step - accuracy: 0.6448 - loss: 0.2656 - val_accuracy: 0.7821 - val_loss: 0.1764
Epoch 2/500
3430/3430 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.7926 - loss: 0.1760 - val_accuracy: 0.8086 - val_loss: 0.1412
Epoch 3/500
1921/3430 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.8083 - loss: 0.1584

KeyboardInterrupt: 

In [ ]:
tf.keras.models.save_model(three_body_model, 'myNN.keras')

In [ ]:
test_x,test_y = get_data(5)

In [ ]:
predictions = three_body_model.predict(test_x)
#validating on training set simply for proof of concept

In [ ]:
#trajectory plot of custom model
pred_traj = get_trajectories(predictions)
ax = plot_trajectories(pred_traj[0],pred_traj[1],pred_traj[2])

In [ ]:
#trajectory plot of true dataset from the Brutus solver
true_traj = get_trajectories(test_y)
bx = plot_trajectories(true_traj[0],true_traj[1],true_traj[2])

In [ ]:
model_preds = pre_trained_model.predict(test_x)

In [ ]:
#trajectory plot of the pre defined model by Breen et. al.
model_traj = get_trajectories(model_preds)
cx = plot_trajectories(model_traj[0],model_traj[1],model_traj[2])

## Optional: Part 2 (extra task)
* Create a helper function that extracts the velocity at each time step.
* Compute the potential energy and the kinetic energy at each time step. The units are chosen such that the total energy is just the sum of these two.
* Check for the conservation of energy in: (i) the data generated by `Brutus` (note that the velocities are given in the last few columns of the data file); (ii) the pre-trained model from the paper; (iii) your ANN model.
* Create a custom loss function and re-train your model to be trained (constrained) by energy conservation. (Unfortunately, you should not expect any significant improvement in the model with the energy-conserving loss function.)
* Reproduce Fig. 6 (with the green line being replaced by your energy-conserving ANN). 

### Comments and hints

* You should probably define helper functions that compute potential energies, velocities and kinetic energies for points along a trajectory. Note that the masses start from rest.
* In order to construct a `tensorflow` custom loss you will need `@tf.function()` versions of the same helper functions. Within these you can only use `tensforflow` methods such as: `tf.concat()` and `tf.math.sqrt()` (i.e., `numpy` methods cannot be used for these functions). 
* See (suggested) `tensorflow` helper functions docstrings below. 
* You can define a **custom loss function** that assumes a batch size that is equal to the number of time steps in a trajectory. You should also make sure that the data is not shuffled so that a **batch always corresponds to a trajectory**. Then you can use a **global time array** to be used to extract velocities and therefore kinetic energies.
* See a suggested structure of the custom loss below. You might get problems with very large kinetic and potential energies for trajectories with close encounters. A suggestion is to clip very large values of `energy_error` in the return statement of the custom energy loss by `tf.clip_by_value(energy_error, -1e6, 1e6)`. 
* The custom loss is then used in `model.compile()` via the argument `loss=custom_energy_loss`.

#### Helper function docstrings
The following are just suggestions. You are allowed to construct a solution using your own helper functions.

In [ ]:
import tensorflow as tf
from tensorflow import keras

@tf.function()
def tf_ediff1d(a):
    """
    TensorFlow equivalent of 1D difference.
    
    Args:
        a (array): A TensorFlow array of dimension (N, 1)
        
    Returns:
        diff (array): An array which gives the difference of successive elements of a.
    """
    raise NotImplementedError

@tf.function()
def tf_compute_velocities(t, p):
    """
    Computes the velocities of the particles from the trajectories starting from rest.
    
    Args:
        t (array[float]): An array of shape (N, 1) giving time steps.
        p (array): An array of dimension (N, 2) where N is the number 
                   of time steps and the two columns give the coordinates
                   of the particle.
                             
    Returns:
        v (array): An array of dimension (N, 2) where N is the number 
                   of time steps and the two columns for each array give 
                   the x and y components for the instantaneous velocity
                   of the particle.
    """
    raise NotImplementedError

@tf.function()
def tf_compute_kinetic_energy(v):
    """
    Computes the kinetic energy for the given velocity vectors
    
    Args:
        v (array): A (N,2) array of veolcities for N time steps.
        
    Returns:
        ke (array): An array of shape (N, 1) giving the kinetic energies at each time step.
    """
    raise NotImplementedError

@tf.function()
def tf_compute_potential_energy(p1, p2, p3):
    """
    Computes the potential energy for the given position vectors. The value of the 
    gravitational constant is taken as 1 (G=1). The masses are the same value (m=1)
    
    Args:
        p1, p2, p3 (tuple of arrays): Three arrays of dimensions (N, 2) where N is the number 
                             of time steps and the two columns for each array give 
                             the positions of the three particles (p1, p2, p3)
                             
    Returns
        pe (array): An array of shape(N, 1) giving the potential energy at each time step
        
    """
    raise NotImplementedError

In [ ]:
# Suggested structure of the custom loss function
tlist = ... # Which will be used in the custom loss

def custom_loss(y, y_pred):
    """
    A custom loss function computing error in energy conservation.
    """
    predicted_positions = ... # Obtained from y_pred
    predicted_velocities = ... # Obtained from predicted_positions, tlist

    initial_potential_energy = ... # Obtained from positions (y)

    ke_predicted_trajectory = ...# Obtained from predicted_velocities
    pe_predicted_trajectory = ... # Obtained from predicted_positions

    error = (ke_predicted_trajectory + pe_predicted_trajectory -
                 initial_potential_energy)

    energy_loss = tf.reduce_mean(tf.abs(error))
    # The relative weight of the two terms in the custom loss might be tuned.
    return tf.keras.losses.MeanAbsoluteError()(y, y_pred) + 0.001*energy_loss